# Tail calls and tail recursion

*Unlike some languages, Python does not treat tail calls differently from other function calls, but the concept is still meaningful and useful.*

A function call `f(...)` whose result is immediately returned is a *tail call*:

```python
return f(...)
```

If any operation, no matter how trivial, ever occurs between receiving the result and returning it, then the call is not a tail call. So the call to `f` is not a tail call here:

```python
return 1 + f(...)  # The last operation was +, not the call to f.
```

Likewise, even the call to `f` in `return f(...)` is not a tail call when that statement appears in:

- a `try` block that has an associated `finally` block, *or*
- a `with` statement.

***Learning check 1:*** Why does that prevent it from being a tail call?

When a void function calls another void function and then immediately returns, that is also a tail call. Python doesn't have void functions, but if `f` never explicitly returns a value, then `f(...)` always evaluates to `None`, so the function call in

```python
f(...)
return  # Or an implicit return by falling off the end.
```

can be regarded as a tail call, in that it always has the same effect as:

```python
return f(...)
```

A *tail-recursive* function is a recursive function whose recursive calls are all tail calls.

A language is said to have *proper tail calls* if, whenever a tail call is made, the caller's stack frame is replaced by the callee's stack frame, so that when $A$ makes a non-tail call to $B$, and $B$ makes a tail call to $C$, $C$ returns directly to $A$. Likewise, if $A$ makes a non-tail call to $B$, and $B$ makes a tail call to $C$, which makes a tail call to $D$, which makes a tail call to $E$, $E$ returns directly to $A$. This holds both when the functions are the same (i.e., in tail recursion) and when they are different. Thus, in languages with proper tail calls, even very "deep" chains of tail calls use only $\mathcal{O}(1)$ call-stack space and don't overflow.

***Learning check 2:*** What is it about tail calls that makes it possible&mdash;in the sense of preserving correct behavior&mdash;for the caller's stack frame to be overwritten, or discarded and replaced, with the callee's stack frame? (This is the most fundamental thing to understand about tail calls&mdash;more so than any definition.)

**Python does *not* have proper tail calls**, so functions like `recursion.semifactorial_tail` and `fibonacci.fibonacci_tail` still raise `RecursionError` for large `n`.

Even when using languages without proper tail calls, like Python, recognizing tail recursion is nonetheless of practical value, because it offers insights into how some algorithms work, and it presents opportunities for manual optimization. In particular, **tail recursion can be converted to iteration without requiring a stack**. It's obvious that, for example, semifactorials can be computed iteratively; but in less straightforward situations, especially if some recursive calls are tail calls and others aren't, some major optimization opportunities can be hard to identify without tail recursion in mind.

When a unary function that is natural to implement recursively with no helper function is implemented tail-recursively, it is often necessary to use a helper function. The helper function's additional parameters correspond to the non-parameter variables an iterative implementation would introduce to maintain state across iterations.

For example, consider summing the elements of a singly linked list with the iterative accumulator pattern:

```python
def sll_sum(head):
    acc = 0
    while head:
        acc += head.value
        head = head.next
    return acc
```

Implemented tail-recursively, the accumulator pattern looks like:

```python
def sll_sum(head):
    def accumulate(acc, node):
        return accumulate(acc + node.value, node.next) if node else acc

    return accumulate(0, head)
```

(Here, the helper function `accumulate` doesn't capture any variables, and therefore could just as reasonably be implemented as a top-level nonpublic function.)

Contrast this to the non-tail recursive strategy:
    
```python
def sll_sum(head):
    return head.value + sll_sum(head.next) if head else 0
```

***Learning check 3:*** Of all the recursive functions you've already implemented in `recursion.py`, which are tail recursive and which are not?